In [2]:
!pip install transformers deepface sentence-transformers opencv-python

import os
import numpy as np
import pandas as pd
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import cv2
from deepface import DeepFace
import torch
from sklearn.metrics.pairwise import cosine_similarity
from google.colab.patches import cv2_imshow
from google.colab.output import eval_js
from google.colab import files
from IPython.display import display, Javascript, Image, clear_output
from base64 import b64decode, b64encode
import io
import threading
import time
import PIL

!pip install python-docx PyPDF2 pytesseract Pillow
!apt-get install tesseract-ocr
!pip install pytesseract

import docx
import PyPDF2
import pytesseract
from PIL import Image
import html


# ======== DOCUMENT GRADING SYSTEM ========

class AssignmentGrader:
    def __init__(self):

        self.tokenizer = AutoTokenizer.from_pretrained("roberta-base")
        self.model = AutoModelForSequenceClassification.from_pretrained("roberta-base")
        self.summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        self.nlp = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")


        from sentence_transformers import SentenceTransformer
        self.semantic_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')



    def extract_text_from_document(self, file_path):
        """Extract text from various document types including images"""

        file_extension = file_path.split('.')[-1].lower()

        try:

            if file_extension == 'txt':
                with open(file_path, 'r', encoding='utf-8', errors='replace') as file:
                    return file.read()


            elif file_extension in ['docx', 'doc']:
                try:
                    doc = docx.Document(file_path)
                    full_text = []

                    for para in doc.paragraphs:
                        full_text.append(para.text)

                    for table in doc.tables:
                        for row in table.rows:
                            for cell in row.cells:
                                full_text.append(cell.text)
                    return '\n'.join(full_text)
                except Exception as e:
                    print(f"Error parsing Word document: {e}")
                    return ""


            elif file_extension == 'pdf':
                try:
                    with open(file_path, 'rb') as file:
                        pdf_reader = PyPDF2.PdfReader(file)
                        text = []
                        for page_num in range(len(pdf_reader.pages)):
                            page = pdf_reader.pages[page_num]
                            text.append(page.extract_text())
                        return '\n'.join(text)
                except Exception as e:
                    print(f"Error parsing PDF: {e}")
                    return ""


            elif file_extension in ['png', 'jpg', 'jpeg', 'bmp', 'tiff', 'webp']:
                try:

                    image = Image.open(file_path)

                    text = pytesseract.image_to_string(image)
                    return text
                except Exception as e:
                    print(f"Error extracting text from image: {e}")
                    return ""


            else:
                print(f"Unsupported file type: {file_extension}")
                print("Attempting to read as plain text...")
                try:
                    with open(file_path, 'r', encoding='utf-8', errors='replace') as file:
                        return file.read()
                except:
                    print("Failed to extract text. Please upload a supported file format.")
                    return ""

        except Exception as e:
            print(f"Error processing file: {e}")
            return ""

    def parse_rubric(self, rubric_text):
        """Parse teacher's rubric/notes into assessment criteria"""

        criteria = {}
        lines = rubric_text.strip().split('\n')
        for line in lines:
            if ':' in line:
                key, value = line.split(':', 1)
                criteria[key.strip()] = value.strip()
        return criteria

    def evaluate_content_against_criteria(self, student_text, criteria):
        """Compare student content against rubric criteria using semantic similarity"""
        scores = {}
        student_embedding = self.semantic_model.encode(student_text)

        for criterion_name, criterion_desc in criteria.items():
            criterion_embedding = self.semantic_model.encode(criterion_desc)
            similarity = cosine_similarity(
                [student_embedding],
                [criterion_embedding]
            )[0][0]
            scores[criterion_name] = similarity

        return scores

    def check_grammar_and_structure(self, text):
        """Check grammar and structure using pre-trained model"""

        sentences = text.split('.')
        errors = []
        for sentence in sentences:
            if len(sentence.strip()) > 5:
                sentiment = self.nlp(sentence)


        structure_score = min(1.0, len(text.split('\n\n')) / 5)
        return {
            "grammar_score": 0.8,
            "structure_score": structure_score,
            "errors": errors
        }

    def generate_feedback(self, content_scores, grammar_results, student_text):
        """Generate personalized feedback based on assessment"""

        summary = ""
        if len(student_text) > 200:
            try:

                max_length = min(100, len(student_text) // 3)
                min_length = min(30, max_length - 10)

                if max_length > min_length:
                    summary = self.summarizer(
                        student_text[:1024],
                        max_length=max_length,
                        min_length=min_length
                    )[0]['summary_text']
            except Exception as e:
                print(f"Summarization error: {e}")
                summary = ""

        summary_intro = f"Summary of your work: {summary}\n\n" if summary else ""


        feedback = [summary_intro] if summary_intro else []
        feedback.append("Strengths:")


        if content_scores:
            strengths = sorted(content_scores.items(), key=lambda x: x[1], reverse=True)[:2]
            for criterion, score in strengths:
                feedback.append(f"• {criterion}: Your work shows strong understanding in this area.")
        else:
            feedback.append("• Unable to determine specific strengths due to text extraction issues.")

        feedback.append("\nAreas for improvement:")

        if content_scores:
            weaknesses = sorted(content_scores.items(), key=lambda x: x[1])[:2]
            for criterion, score in weaknesses:
                feedback.append(f"• {criterion}: Consider enhancing your work by focusing more on this aspect.")
        else:
            feedback.append("• Unable to determine specific improvement areas due to text extraction issues.")


        if grammar_results["grammar_score"] < 0.7:
            feedback.append("• Check your grammar and sentence structure for clarity.")

        if grammar_results["structure_score"] < 0.6:
            feedback.append("• Consider improving paragraph organization for better flow.")

        return "\n".join(feedback)
    def calculate_grade(self, content_scores, grammar_results):
        """Calculate overall grade based on all assessments"""
        content_weight = 0.7
        grammar_weight = 0.2
        structure_weight = 0.1

        avg_content_score = sum(content_scores.values()) / len(content_scores)

        final_score = (
            avg_content_score * content_weight +
            grammar_results["grammar_score"] * grammar_weight +
            grammar_results["structure_score"] * structure_weight
        )


        if final_score >= 0.9:
            letter_grade = "A"
        elif final_score >= 0.8:
            letter_grade = "B"
        elif final_score >= 0.7:
            letter_grade = "C"
        elif final_score >= 0.6:
            letter_grade = "D"
        else:
            letter_grade = "F"

        return {
            "numerical_score": round(final_score * 100, 1),
            "letter_grade": letter_grade
        }

    def grade_assignment(self, student_file, rubric_file):
        """Main function to grade an assignment"""
        student_text = self.extract_text_from_document(student_file)
        rubric_text = self.extract_text_from_document(rubric_file)

        criteria = self.parse_rubric(rubric_text)
        content_scores = self.evaluate_content_against_criteria(student_text, criteria)
        grammar_results = self.check_grammar_and_structure(student_text)

        feedback = self.generate_feedback(content_scores, grammar_results, student_text)
        grade = self.calculate_grade(content_scores, grammar_results)

        return {
            "grade": grade,
            "feedback": feedback,
            "content_scores": content_scores,
            "grammar_results": grammar_results
        }


# ======== EMOTION DETECTION SYSTEM ========

class EmotionDetector:
    def __init__(self):

        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


        self.emotions = ["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"]

    def detect_emotions_from_frame(self, frame):
        """Detect emotions from a video frame"""

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = self.face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30)
        )

        results = []
        for (x, y, w, h) in faces:
            face_region = frame[y:y+h, x:x+w]

            try:

                analysis = DeepFace.analyze(
                    face_region,
                    actions=['emotion'],
                    enforce_detection=False
                )

                emotion = analysis[0]['dominant_emotion']
                emotion_scores = analysis[0]['emotion']

                results.append({
                    'position': (x, y, w, h),
                    'dominant_emotion': emotion,
                    'emotion_scores': emotion_scores
                })

            except Exception as e:

                print(f"Error in emotion detection: {e}")
                continue

        return results

    def process_uploaded_image(self, image_path):
        """Process an uploaded image for emotion detection"""
        frame = cv2.imread(image_path)
        if frame is None:
            return {"error": "Failed to load image"}

        emotion_results = self.detect_emotions_from_frame(frame)


        for result in emotion_results:
            x, y, w, h = result['position']
            emotion = result['dominant_emotion']

            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)


        cv2_imshow(frame)

        return emotion_results

    def analyze_classroom_emotions(self, frame):
        """Analyze emotions across the classroom"""
        emotion_results = self.detect_emotions_from_frame(frame)


        emotion_counts = {emotion: 0 for emotion in self.emotions}
        for result in emotion_results:
            emotion_counts[result['dominant_emotion']] += 1


        engaged_emotions = ['happy', 'surprise']
        disengaged_emotions = ['sad', 'angry', 'disgust']
        neutral_emotions = ['neutral', 'fear']

        total_faces = len(emotion_results)
        if total_faces == 0:
            return {
                'emotion_distribution': emotion_counts,
                'engagement_score': 0,
                'dominant_mood': 'unknown'
            }

        engaged_count = sum(emotion_counts[e] for e in engaged_emotions)
        disengaged_count = sum(emotion_counts[e] for e in disengaged_emotions)

        engagement_score = (engaged_count - disengaged_count) / total_faces


        dominant_mood = max(emotion_counts.items(), key=lambda x: x[1])[0] if emotion_counts else 'unknown'

        return {
            'emotion_distribution': emotion_counts,
            'engagement_score': engagement_score,
            'dominant_mood': dominant_mood
        }

# ======== COLAB CAMERA INTEGRATION ========



def take_photo(filename='photo.jpg', quality=0.8):
    """Use JavaScript to capture an image from the webcam"""
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();


      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);


      await new Promise((resolve) => {
        capture.onclick = resolve;
      });

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename


def get_webcam_frame():
    """Capture a frame from the webcam using Colab's JavaScript interface"""
    from IPython.display import display, Javascript
    from google.colab.output import eval_js

    js_code = '''
    async function captureFrame() {
      const video = document.createElement('video');
      video.style.display = 'none';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      document.body.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Wait a bit for camera to initialize
      await new Promise(r => setTimeout(r, 1000));

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      video.remove();

      return canvas.toDataURL('image/jpeg', 0.8);
    }
    '''

    display(Javascript(js_code))
    try:
        data = eval_js('captureFrame()')
        img = js_to_image(data)
        return img
    except Exception as e:
        print(f"Error capturing frame: {e}")
        return None

# ======== REAl TIME EMOTION DETECTION ========

face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640;
      captureCanvas.height = 480;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data



def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """

  image_bytes = b64decode(js_reply.split(',')[1])

  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)

  img = cv2.imdecode(jpg_as_np, flags=1)

  return img


def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """

  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()

  bbox_PIL.save(iobuf, format='png')

  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes


def face_detect():
  import cv2
  from PIL import Image
  import numpy as np
  import os
  from keras.models import load_model
  from time import sleep
  from keras.preprocessing.image import img_to_array
  from keras.preprocessing import image

  video_stream()

  label_html = 'Capturing...'

  bbox = ''
  count = 0
  face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') # Face Detection
  classifier =load_model('/content/model_78.h5')  #Load model
  emotion_labels = ['Angry','Disgust','Fear','Happy','Neutral', 'Sad', 'Surprise']

  while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
      break


    img = js_to_image(js_reply["img"])


    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


    faces = face_cascade.detectMultiScale(gray)

    for (x,y,w,h) in faces:
      bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(0,255,0),2)
      roi_gray = gray[y:y+h,x:x+w]
      roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
      if np.sum([roi_gray])!=0:
        roi = roi_gray.astype('float')/255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi,axis=0)
        prediction = classifier.predict(roi)[0]   #Prediction
        label=emotion_labels[prediction.argmax()]
        label_position = (x,y)
        cv2.putText(bbox_array,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)   # Text Adding
        label_html = 'Capturing...' + label
      else:
        cv2.putText(bbox_array,'No Faces',(30,80),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)





    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    bbox_bytes = bbox_to_bytes(bbox_array)
    bbox = bbox_bytes


# ======== INTEGRATED SYSTEM ========

class EducationalAssistant:
    def __init__(self):
        self.grader = AssignmentGrader()
        self.emotion_detector = EmotionDetector()


    def process_assignment(self, student_file, rubric_file):
        """Process a student assignment with grading and feedback"""
        return self.grader.grade_assignment(student_file, rubric_file)

    def upload_and_process_files(self):
        """Upload student assignment and rubric files through Colab interface"""
        print("Upload student assignment file (txt, docx, pdf, jpg, png, etc.):")
        print("This can be an essay, report, assignment, or even an image containing text.")
        uploaded_student = files.upload()
        student_file = list(uploaded_student.keys())[0]

        print("Upload teacher rubric file (txt, docx, pdf, etc.):")
        print("This should contain evaluation criteria, typically in format 'Criterion: Description'")
        uploaded_rubric = files.upload()
        rubric_file = list(uploaded_rubric.keys())[0]

        return self.process_assignment(student_file, rubric_file)

    def process_uploaded_image_for_emotions(self):
        """Upload and process an image for emotion detection"""
        print("Upload classroom image for emotion detection:")
        uploaded_images = files.upload()
        image_file = list(uploaded_images.keys())[0]

        results = self.emotion_detector.process_uploaded_image(image_file)
        emotion_data = self.emotion_detector.analyze_classroom_emotions(cv2.imread(image_file))

        classroom_report = self.generate_classroom_report(emotion_data)
        print(classroom_report)

        return results

    def monitor_classroom_using_colab_camera(self, duration=30):
        """Use Colab's camera for classroom monitoring"""
        print("Starting classroom monitoring...")
        print("This will capture an image from your webcam every 5 seconds and analyze emotions.")
        print(f"Monitoring will run for {duration} seconds.")

        end_time = time.time() + duration

        while time.time() < end_time:
            print("Capturing frame...")
            try:

                photo_filename = take_photo()
                frame = cv2.imread(photo_filename)

                if frame is None:
                    print("Failed to capture frame. Trying again...")
                    continue

                emotion_results = self.emotion_detector.detect_emotions_from_frame(frame)


                for result in emotion_results:
                    x, y, w, h = result['position']
                    emotion = result['dominant_emotion']

                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)


                cv2_imshow(frame)

                emotion_data = self.emotion_detector.analyze_classroom_emotions(frame)
                report = self.generate_classroom_report(emotion_data)
                print(report)


                time.sleep(5)

            except Exception as e:
                print(f"Error in monitoring: {e}")
                time.sleep(1)

        print("Classroom monitoring completed.")

    def generate_classroom_report(self, emotion_data):
        """Generate report on classroom emotions"""
        report = ["## Classroom Emotional Analysis Report"]

        report.append("\n### Emotion Distribution:")
        for emotion, count in emotion_data['emotion_distribution'].items():
            report.append(f"- {emotion.capitalize()}: {count} students")

        report.append(f"\n### Engagement Score: {emotion_data['engagement_score']:.2f}")
        report.append(f"(-1.0 = completely disengaged, 1.0 = fully engaged)")

        report.append(f"\n### Dominant Mood: {emotion_data['dominant_mood'].capitalize()}")

        report.append("\n### Recommendations:")
        if emotion_data['engagement_score'] < -0.3:
            report.append("- Consider an activity change to re-engage students")
            report.append("- Check if the material might be too challenging or not challenging enough")
        elif emotion_data['dominant_mood'] in ['sad', 'angry']:
            report.append("- Students appear distressed - consider checking in or providing a short break")
        elif emotion_data['dominant_mood'] == 'surprise':
            report.append("- Students appear stimulated - good time for introducing new concepts")

        return "\n".join(report)



print("Educational Assistant System")
print("===========================")
print("1. Initialize the system")
assistant = EducationalAssistant()
print("System initialized successfully!")

print("\nYou can use the following functions:")
print("1. assistant.upload_and_process_files() - For grading assignments")
print("2. assistant.process_uploaded_image_for_emotions() - For analyzing emotions in a single image")
print("3. assistant.monitor_classroom_using_colab_camera() - For live classroom monitoring (30 seconds)")
print("4. face_detect() - For live classroom monitoring at real time")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Educational Assistant System
1. Initialize the system


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Device set to use cpu


System initialized successfully!

You can use the following functions:
1. assistant.upload_and_process_files() - For grading assignments
2. assistant.process_uploaded_image_for_emotions() - For analyzing emotions in a single image
3. assistant.monitor_classroom_using_colab_camera() - For live classroom monitoring (30 seconds)
4. face_detect() - For live classroom monitoring at real time


In [4]:
def run_interactive_menu():
    while True:
        print("\n===== Educational Assistant Menu =====")
        print("1. Grade assignments")
        print("2. Analyze emotions in uploaded image")
        print("3. Live classroom monitoring (30 seconds)")
        print("4. Exit")

        choice = input("Enter your choice (1-4): ")

        if choice == '1':
            result = assistant.upload_and_process_files()
            print("\n=== Grading Results ===")
            print(f"Grade: {result['grade']['letter_grade']} ({result['grade']['numerical_score']}%)")
            print("\nFeedback:")
            print(result['feedback'])

        elif choice == '2':
            assistant.process_uploaded_image_for_emotions()

        elif choice == '3':
            duration = 30
            try:
                duration_input = input("Enter monitoring duration in seconds (default: 30): ")
                if duration_input.strip():
                    duration = int(duration_input)
            except ValueError:
                print("Invalid input, using default 30 seconds")

            assistant.monitor_classroom_using_colab_camera(duration=duration)

        elif choice == '4':
            print("Exiting Educational Assistant. Goodbye!")
            break

        else:
            print("Invalid choice. Please enter a number between 1 and 4.")

# Run the interactive menu
run_interactive_menu()


===== Educational Assistant Menu =====
1. Grade assignments
2. Analyze emotions in uploaded image
3. Live classroom monitoring (30 seconds)
4. Exit
Enter your choice (1-4): 1
Upload student assignment file (txt, docx, pdf, jpg, png, etc.):
This can be an essay, report, assignment, or even an image containing text.


Saving Title.docx to Title.docx
Upload teacher rubric file (txt, docx, pdf, etc.):
This should contain evaluation criteria, typically in format 'Criterion: Description'


Saving Title.docx to Title (1).docx

=== Grading Results ===
Grade: C (73.0%)

Feedback:
Summary of your work: Computers have become an integral part of our daily lives, revolutionizing the way we work, communicate, learn, and entertain ourselves. From personal use to industrial applications, computers have dramatically changed the face of the modern world.


Strengths:
• Title: Your work shows strong understanding in this area.

Areas for improvement:
• Title: Consider enhancing your work by focusing more on this aspect.
• Consider improving paragraph organization for better flow.

===== Educational Assistant Menu =====
1. Grade assignments
2. Analyze emotions in uploaded image
3. Live classroom monitoring (30 seconds)
4. Exit
Enter your choice (1-4): 4
Exiting Educational Assistant. Goodbye!


In [ ]:
face_detect()

<IPython.core.display.Javascript object>

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/model_by_nishant.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)